In [1]:
# save
import geopandas as gpd
import osmnx as ox
import networkx as nx
from shapely.geometry import Point


In [4]:
random_points_gdf = gpd.read_file('/Users/angel_zhong/Documents/GitHub/park-network-distance/data/countyRandomPoint/CountyRandomPoint.shp')
park_points_gdf = gpd.read_file('/Users/angel_zhong/Documents/GitHub/park-network-distance/data/parkPoint/park3_FeatureToPoint.shp')
county_gdf = gpd.read_file('/Users/angel_zhong/Documents/GitHub/park-network-distance/data/tl_2020_us_county/tl_2020_us_county.shp')


In [3]:
random_points_gdf.head()


,CID,geometry
0,0,POINT (892412.278 1090889.261)
1,0,POINT (855626.104 1081505.318)
2,0,POINT (853773.624 1087400.239)
3,0,POINT (886827.950 1106448.313)
4,0,POINT (861846.327 1095838.366)


In [14]:
# Check the spatial references of both GeoDataFrames
print("Random points CRS:", random_points_gdf.crs)
print("park points CRS:", park_points_gdf.crs)
print("County boundaries CRS:", county_gdf.crs)

Random points CRS: EPSG:4326
park points CRS: EPSG:4326
County boundaries CRS: EPSG:4326


In [13]:
target_crs = 'EPSG:4326'

if random_points_gdf.crs != target_crs:
    random_points_gdf = random_points_gdf.to_crs(target_crs)

if park_points_gdf.crs != target_crs:
    park_points_gdf = park_points_gdf.to_crs(target_crs)

if county_gdf.crs != target_crs:
    county_gdf = county_gdf.to_crs(target_crs)

In [15]:
random_points_with_county = gpd.sjoin(random_points_gdf, county_gdf, op='within')
park_points_with_county = gpd.sjoin(park_points_gdf, county_gdf, op='within')


/Users/angel_zhong/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3400: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/Users/angel_zhong/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3400: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [9]:
random_points_with_county.head()

,CID,geometry,index_right,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON
0,0,POINT (-86.43654 32.48595),1327,01,001,00161526,01001,Autauga,Autauga County,06,H1,G4020,388,33860,None,A,1539634184,25674812,+32.5322367,-086.6464395
1,0,POINT (-86.83793 32.43465),1327,01,001,00161526,01001,Autauga,Autauga County,06,H1,G4020,388,33860,None,A,1539634184,25674812,+32.5322367,-086.6464395
2,0,POINT (-86.85158 32.48887),1327,01,001,00161526,01001,Autauga,Autauga County,06,H1,G4020,388,33860,None,A,1539634184,25674812,+32.5322367,-086.6464395
3,0,POINT (-86.47934 32.62978),1327,01,001,00161526,01001,Autauga,Autauga County,06,H1,G4020,388,33860,None,A,1539634184,25674812,+32.5322367,-086.6464395
4,0,POINT (-86.75686 32.55719),1327,01,001,00161526,01001,Autauga,Autauga County,06,H1,G4020,388,33860,None,A,1539634184,25674812,+32.5322367,-086.6464395


In [10]:
len(random_points_with_county)

201695

In [11]:
len(random_points_gdf)

201695

In [16]:
def process_county(county, random_points_with_county, park_points_with_county, output_dict):
    print(f"Processing county: {county.NAME}")

    # Get random points and park points within the current county
    random_points_in_county = random_points_with_county[random_points_with_county['index_right'] == county.Index]
    park_points_in_county = park_points_with_county[park_points_with_county['index_right'] == county.Index]

    # Download the street network within the county boundary
    county_boundary = county.geometry
    G = ox.graph_from_polygon(county_boundary, network_type='all')
    G = ox.add_edge_speeds(G)
    G = ox.add_edge_travel_times(G)

    # Get the nearest network nodes for random points and park points
    random_points_nodes = ox.distance.nearest_nodes(G, random_points_in_county.geometry, return_dist=False)
    park_points_nodes = ox.distance.nearest_nodes(G, park_points_in_county.geometry, return_dist=False)

    # Add 'osmid' column to the dataframes
    random_points_in_county['osmid'] = random_points_nodes
    park_points_in_county['osmid'] = park_points_nodes

    # Find the closest park for each random point
    closest_parks = []
    for random_point in random_points_in_county.itertuples():
        closest_park = find_closest_park(random_point, park_points_in_county, G)
        print(f"Closest park to random point {random_point.Index} is park {closest_park.Index}")
        closest_parks.append((random_point.Index, closest_park.Index))

    output_dict[county.Index] = closest_parks


In [19]:
import concurrent.futures

output_dict = {}
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = []
    for county in county_gdf.itertuples():
        future = executor.submit(process_county, county, random_points_with_county, park_points_with_county, output_dict)
        futures.append(future)

    for future in concurrent.futures.as_completed(futures):
        print(f"Finished processing county: {future.result()}")

: 

In [1]:
output_dict

NameError: name 'output_dict' is not defined